In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("banking_analytics").master('local[*]').getOrCreate()

credit_db = spark.read.csv('/home/kurinchiban/Desktop/banking-analytics-with-pyspark/datasets/credit_card.csv', header=True, inferSchema=True)


25/04/03 19:40:46 WARN SizeEstimator: Failed to check whether UseCompressedOops is set; assuming yes


What is the distribution of customers based on age and geography?
Are there any regions with higher numbers of young or older customers?

In [5]:
from pyspark.sql import functions as F
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

age_ranges = [
    (18, 30, "Young"),
    (31, 50, "Middle-aged"),
    (51, 100, "Senior")
]

def categorize_age(age):
    for min_age, max_age, category in age_ranges:
        if min_age <= age <= max_age:
            return category
    return "Unknown"

categorize_age_udf = udf(categorize_age, StringType())

df_with_age_group = credit_db.withColumn("AgeGroup", categorize_age_udf(credit_db['Age']))

age_group_distribution_by_geography = df_with_age_group.groupBy('Geography', 'AgeGroup').agg(
    F.count('CustomerId').alias('CustomerCount')
)

age_group_distribution_by_geography.show()


max_customer_row = age_group_distribution_by_geography.orderBy(F.desc('CustomerCount')).first()

print(f'The {max_customer_row[0]} is having lot of {max_customer_row[1]} peoples')

+---------+-----------+-------------+
|Geography|   AgeGroup|CustomerCount|
+---------+-----------+-------------+
|   France|      Young|         1038|
|    Spain|Middle-aged|         1693|
|    Spain|      Young|          477|
|   France|     Senior|          598|
|  Germany|     Senior|          356|
|   France|Middle-aged|         3378|
|  Germany|Middle-aged|         1700|
|  Germany|      Young|          453|
|    Spain|     Senior|          307|
+---------+-----------+-------------+

The France is having lot of Middle-aged peoples


Write a PySpark query to filter out all active customers (IsActiveMember = 1) and return the relevant customer details.

In [6]:
def categorize_member(value):
    if value == 1:
        return "ActiveMember"
    else:
        return "NotActiveMember"
    
udf_categorize_age = udf(categorize_member, StringType())

active_member_df = credit_db.withColumn('MemberStatus',udf_categorize_age(credit_db['IsActiveMember']))

member_status = active_member_df.groupBy('MemberStatus').agg(F.count("*").alias("MemberStatusCount"))

member_status.show()

+---------------+-----------------+
|   MemberStatus|MemberStatusCount|
+---------------+-----------------+
|NotActiveMember|             4849|
|   ActiveMember|             5151|
+---------------+-----------------+



Write a PySpark query to find all customers with a credit score greater than 700, and return their CustomerId, Surname, CreditScore, and Age.

In [49]:
filtered_df = credit_db.filter(F.col("CreditScore") > 700)

result_df = filtered_df.select("CustomerId", "Surname", "CreditScore", "Age")

result_df.show()

+----------+----------+-----------+---+
|CustomerId|   Surname|CreditScore|Age|
+----------+----------+-----------+---+
|  15737888|  Mitchell|        850| 43|
|  15592531|  Bartlett|        822| 50|
|  15568982|       Hao|        726| 24|
|  15577657|  McDonald|        732| 41|
|  15625047|       Yen|        846| 38|
|  15736816|     Young|        756| 36|
|  15732963|  Clements|        722| 29|
|  15729599|   Lorenzo|        804| 33|
|  15717426| Armstrong|        850| 36|
|  15755196|    Lavine|        834| 49|
|  15754849|     Tyler|        776| 32|
|  15602280|    Martin|        829| 27|
|  15771873|    Buccho|        776| 37|
|  15683553|   O'Brien|        788| 33|
|  15647091|  Endrizzi|        725| 19|
|  15651280|    Hunter|        742| 35|
|  15789484|   Hammond|        751| 36|
|  15641582|   Chibugo|        735| 43|
|  15703793|Konovalova|        738| 58|
|  15620344|     McKee|        813| 29|
+----------+----------+-----------+---+
only showing top 20 rows



Group the customers by geography and calculate the average credit Score for each geography.

In [9]:
geography_df = credit_db.groupBy('Geography').agg(F.round(F.avg(F.col("CreditScore")),2).alias('average_income')).show()

+---------+--------------+
|Geography|average_income|
+---------+--------------+
|  Germany|        651.45|
|   France|        649.67|
|    Spain|        651.33|
+---------+--------------+

